<font size="8"> **Estimating krill growth potential from ACCESS-OM2-01 outputs** </font>  
In this notebook, we will estimate krill growth potential from the ACCESS-OM2-01 model outputs as a proxy for good quality krill habitat. This information gives us an indication of prey availability for crabeater seals. Krill growth potential calculations followed methods described in [Veytia et al 2020](https://doi.org/10.1038/s41558-020-0758-4).

# Setting working directory
In order to ensure these notebooks work correctly, we will set the working directory. We assume that you have saved a copy of this repository in your home directory (represented by `~` in the code chunk below). If you have saved this repository elsewhere in your machine, you need to ensure you update this line with the correct filepath where you saved these notebooks.

In [1]:
import os
os.chdir(os.path.expanduser('~/Chapter2_Crabeaters/Scripts'))

# Loading other relevant libraries

In [2]:
from dask.distributed import Client
from glob import glob
#Accessing model data
import cosima_cookbook as cc
#Useful functions
import UsefulFunctions as uf
#Dealing with data
import xarray as xr
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Paralellising work 

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/42209/status,
Dashboard: /proxy/42209/status,Workers: 7
Total threads: 14,Total memory: 63.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37473,Workers: 7
Dashboard: /proxy/42209/status,Total threads: 14
Started: Just now,Total memory: 63.00 GiB
Comm: tcp://127.0.0.1:40585,Total threads: 2
Dashboard: /proxy/43641/status,Memory: 9.00 GiB
Nanny: tcp://127.0.0.1:41171,


# Setting up
We will create a new `cosima cookbook` session to load the model outputs of interest, and we will also create a dictionary that contains useful information related to data extraction.

In [4]:
#Creating new COSIMA cookbook session
session = cc.database.create_session()

#Creating dictionary with useful information
varDict = {'model': 'ACCESS-OM2-01',
           #ACCESS-OM2-01 cycle 4 (1958-2018)
           'exp': '01deg_jra55v140_iaf_cycle4',
           #Temporal resolution
           'freq': '1 daily',
           #Output folder
           'base_folder_out': '/g/data/v45/la6889/Chapter2_Crabeaters/'}

# Loading SST and phytoplankton data from ACCESS-OM2-01
To calculate krill gross growth potential, we need phytoplankton at the sea surface (`surface_phy`) and sea surface temperature (`surface_pot_temp`). These variables are available in ACCESS-OM2-01, but before using them, we need to correct longitudes so they range between -180 and +180, and convert phytoplankton units from $mmol N \times m^{-3}$ to $mg \times m^{-3}$, and temperature units from Kelvin to $^{\circ}C$.  

In [5]:
#Folders to save phytoplankton and SST
phyto_out = os.path.join(varDict['base_folder_out'], 'Phytoplankton')
SST_out = os.path.join(varDict['base_folder_out'], 'SST')
#Ensure folders exist, if not, create them
os.makedirs(phyto_out, exist_ok = True)
os.makedirs(SST_out, exist_ok = True)

In [16]:
#Years of interest
yrs = np.arange(1981, 2014)

#Loop thorugh years
for yr in yrs:
    #Loading phytoplankton data from fourth cycle
    phyto_df = uf.getACCESSdata_SO('surface_phy', f'{yr}-10', f'{yr+1}-01', 
                                   freq = varDict['freq'], ses = session, 
                                   minlat = -80, maxlat = -55, exp = varDict['exp'], 
                                   ice_data = False)
    #Loading temperature data from fourth cycle
    temp_df = uf.getACCESSdata_SO('surface_pot_temp', f'{yr}-10', f'{yr+1}-01', 
                                  freq = varDict['freq'], ses = session, minlat = -80,
                                   maxlat = -55, exp = varDict['exp'], ice_data = False)
    
    #Transforming longitudes so their range is +/-180 degrees
    phyto_df = uf.corrlong(phyto_df).sel(time = slice(f'{yr}-11', f'{yr}-12'),
                                         xt_ocean = slice(30, 170))
    temp_df = uf.corrlong(temp_df).sel(time = slice(f'{yr}-11', f'{yr}-12'),
                                         xt_ocean = slice(30, 170))
    
    #Converting phytoplankton units from mmol*N/m^-3 to mg/m^-3
    phyto_df = phyto_df*14.0067
    
    #Converting temperature data from Kelvin to degrees C
    temp_df = temp_df-273.15
    #Masking out areas where temperature is above 5C
    temp_df = xr.where(temp_df > 5, np.nan, temp_df)

    #File names
    p_out = os.path.join(phyto_out, f'daily_surf_phyto_Nov-Dec_{yr}.nc')
    t_out = os.path.join(SST_out, f'daily_sst_Nov-Dec_{yr}.nc')

    #Save data arrays
    phyto_df.to_netcdf(p_out)
    temp_df.to_netcdf(t_out)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3

## Loading transformed SST and phytoplankton datasets

In [6]:
phyto_ds = xr.open_mfdataset(sorted(glob(os.path.join(phyto_out, '*.nc')))).surface_phy
sst_ds = xr.open_mfdataset(sorted(glob(os.path.join(SST_out, '*.nc')))).surface_pot_temp

0.3.0


# Calculating krill gross growth potential (GGP)
We estimate Antarctic krill growth potential for November and December of every year during our study period (1981-2013) following methods in [Veytia et al 2020](https://doi.org/10.1038/s41558-020-0758-4).  

In [5]:
#Folders to save krill GGP
krill_out = os.path.join(varDict['base_folder_out'], 'Krill_habitat')
#Ensure folder exists, if not, create it
os.makedirs(krill_out, exist_ok = True)

In [8]:
#Applying calculation every year
for yr, phyto_da in phyto_ds.groupby('time.year'):
    #Select SST for correct year
    sst_da = sst_ds.sel(time = str(yr))
    #Calculate growth rate and GGP
    monthly_growth, ggp_krill = uf.krill_growth(sst_da, phyto_da)
    #Save results
    gr_out = os.path.join(krill_out, f'monthly_krill_growth_rate_Nov-Dec_{yr}.nc')
    ggp_out = os.path.join(krill_out, f'monthly_krill_ggp_Nov-Dec_{yr}.nc')
    #Save data arrays
    monthly_growth.to_netcdf(gr_out)
    ggp_krill.to_netcdf(ggp_out)

## Loading GGP and growth rates

In [6]:
krill_gr = xr.open_mfdataset(sorted(glob(os.path.join(krill_out, '*growth*.nc')))).krill_growth_rate
krill_ggp = xr.open_mfdataset(sorted(glob(os.path.join(krill_out, '*ggp*.nc')))).krill_ggp

0.3.0


# Adding krill data to environmental dataset
Note that this variable was added after models were run. Krill data can only be saved after steps 03 and 04 are completed.

## Loading crabeater data for East Antarctica

In [44]:
crab_file = '../Environmental_Data/model_env_pres_bg_20x_Indian_weaning.csv'
crabeaters = pd.read_csv(crab_file)
crabeaters

,year,month,xt_ocean,yt_ocean,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m,freez_pot_Wm2,...,SSS_psu,bottom_sal_psu,vel_lat_surf_msec,vel_lat_bottom_msec,vel_lon_surf_msec,vel_lon_bottom_msec,lt_pack_ice,dist_ice_edge_km,krill_ggp,krill_growth_rate
0,1998,11,75.85,-69.113,1,89.937,-492.145,130.290,628.583,-541.15330,...,34.448925,34.519634,-0.027874,0.001519,-0.081487,0.013444,0.595238,-1007.829786,0.924939,-3.581085
1,1998,11,75.35,-69.028,1,89.651,-481.137,117.409,729.212,-501.89883,...,34.478040,34.547890,-0.056935,-0.009752,-0.060686,0.011944,0.630952,-996.341423,0.923353,-3.650778
2,1996,11,76.05,-68.986,1,89.957,-479.088,119.262,771.083,-1807.57590,...,34.444534,34.520966,-0.041805,0.018873,-0.024342,0.026100,0.595238,-945.316770,0.931693,-3.283398
3,1998,11,75.45,-68.606,1,89.725,-434.549,71.716,657.067,-372.80582,...,34.473373,34.530651,-0.022065,0.024582,-0.151900,-0.064935,0.630952,-950.011791,0.924193,-3.614363
4,1998,11,75.35,-68.395,1,89.720,-410.830,47.949,578.417,-86.98963,...,34.532246,34.543610,-0.007439,0.032207,-0.101876,-0.026731,0.630952,-926.231709,0.920958,-3.756112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32363,1989,11,74.05,-59.442,0,89.919,543.100,725.785,1618.292,-2010.66210,...,33.729412,34.719143,0.063438,0.001540,0.004965,-0.005750,0.000000,-83.181802,1.050051,2.473643
32364,1998,11,71.75,-59.238,0,89.976,559.524,720.119,4481.736,-8366.12300,...,33.869095,34.632740,-0.000976,0.000412,0.076354,-0.005092,0.000000,76.604135,1.089497,4.686903
32365,1996,11,76.35,-59.238,0,89.961,506.485,705.324,1267.889,-8224.30900,...,33.836685,34.741093,0.015163,0.000069,0.007057,0.003959,0.011905,52.081125,1.098715,5.171012
32366,1989,11,73.95,-58.827,0,89.971,610.227,790.155,2223.069,-4845.99760,...,33.750946,34.673641,0.062015,-0.000036,0.164571,-0.000467,0.000000,-18.242142,1.062114,3.086222


In [40]:
#Getting coordinates from crabeater dataset
lat = xr.DataArray(crabeaters.yt_ocean.values)
lon = xr.DataArray(crabeaters.xt_ocean.values)
#Getting data of observation from the crabeater data
time = xr.DataArray(crabeaters.apply(lambda x: pd.to_datetime(f'{int(x.year)}-{int(x.month)}-16'), axis = 1))

## Defining function to extract data

In [41]:
def extract_data(var_df, lat, lon, time, bio_data):
    #Extracting data
    var_sub = var_df.sel(time = time, yt_ocean = lat, xt_ocean = lon, method = 'nearest')
    
    #Transforming to data frame
    var_pd = var_sub.to_dataframe().sort_values(['time', 'xt_ocean', 'yt_ocean'])
    #Adding year and month
    var_pd['year'] = var_pd.time.dt.year
    var_pd['month'] = var_pd.time.dt.month
    #Removing time column that is no longer needed
    var_pd.drop(columns = 'time', inplace = True)
    #Finding name of columns to round up
    round_cols = [i for i in var_pd.columns if 'ocean' in i]
    #Rounding coordinate values prior to merging
    var_pd = var_pd.round({round_cols[0]: 3, round_cols[1]: 3})
    #Getting column names for merging
    cols = var_pd.drop(columns = var_df.name).columns.tolist()

    #Merge to biological data
    bio_data['id'] = bio_data.index
    bio_data = bio_data.merge(var_pd, on = cols, how = 'left').drop_duplicates()
    bio_data = bio_data.drop(columns = 'id')
    
    #Return results
    return bio_data

## Apply function

In [42]:
crabeaters = extract_data(krill_ggp, lat, lon, time, crabeaters)
crabeaters = extract_data(krill_gr, lat, lon, time, crabeaters)
crabeaters

,year,month,xt_ocean,yt_ocean,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m,freez_pot_Wm2,...,SSS_psu,bottom_sal_psu,vel_lat_surf_msec,vel_lat_bottom_msec,vel_lon_surf_msec,vel_lon_bottom_msec,lt_pack_ice,dist_ice_edge_km,krill_ggp,krill_growth_rate
0,1998,11,75.85,-69.113,1,89.937,-492.145,130.290,628.583,-541.15330,...,34.448925,34.519634,-0.027874,0.001519,-0.081487,0.013444,0.595238,-1007.829786,0.924939,-3.581085
1,1998,11,75.35,-69.028,1,89.651,-481.137,117.409,729.212,-501.89883,...,34.478040,34.547890,-0.056935,-0.009752,-0.060686,0.011944,0.630952,-996.341423,0.923353,-3.650778
2,1996,11,76.05,-68.986,1,89.957,-479.088,119.262,771.083,-1807.57590,...,34.444534,34.520966,-0.041805,0.018873,-0.024342,0.026100,0.595238,-945.316770,0.931693,-3.283398
3,1998,11,75.45,-68.606,1,89.725,-434.549,71.716,657.067,-372.80582,...,34.473373,34.530651,-0.022065,0.024582,-0.151900,-0.064935,0.630952,-950.011791,0.924193,-3.614363
4,1998,11,75.35,-68.395,1,89.720,-410.830,47.949,578.417,-86.98963,...,34.532246,34.543610,-0.007439,0.032207,-0.101876,-0.026731,0.630952,-926.231709,0.920958,-3.756112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40651,1989,11,74.05,-59.442,0,89.919,543.100,725.785,1618.292,-2010.66210,...,33.729412,34.719143,0.063438,0.001540,0.004965,-0.005750,0.000000,-83.181802,1.050051,2.473643
40652,1998,11,71.75,-59.238,0,89.976,559.524,720.119,4481.736,-8366.12300,...,33.869095,34.632740,-0.000976,0.000412,0.076354,-0.005092,0.000000,76.604135,1.089497,4.686903
40653,1996,11,76.35,-59.238,0,89.961,506.485,705.324,1267.889,-8224.30900,...,33.836685,34.741093,0.015163,0.000069,0.007057,0.003959,0.011905,52.081125,1.098715,5.171012
40654,1989,11,73.95,-58.827,0,89.971,610.227,790.155,2223.069,-4845.99760,...,33.750946,34.673641,0.062015,-0.000036,0.164571,-0.000467,0.000000,-18.242142,1.062114,3.086222


## Save results

In [43]:
crabeaters.to_csv(crab_file, index = False)